In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *


spark = SparkSession.builder.getOrCreate()


spark.conf.set(
    "",
    ""
)

# EVENTHUBS_NAMESPACE = "Traffic.servicebus.windows.net"
# EVENT_HUB_NAME = "traffictopic"
# CONNECTION_STRING = ""


# kafka_options = {
#     "kafka.bootstrap.servers": f"{EVENTHUBS_NAMESPACE}:9093",
#     "subscribe": EVENT_HUB_NAME,
#     "kafka.security.protocol": "SASL_SSL",
#     "kafka.sasl.mechanism": "PLAIN",
#     "kafka.sasl.jaas.config": f'kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="{CONNECTION_STRING}";',
#     "startingOffsets": "latest",  
#     "failOnDataLoss": "false"
# }


bronze_path = "abfss://traffic@hellooodb.dfs.core.windows.net/raw_data"
checkpoint_path = bronze_path + "/_checkpoints"

streaming_df = (
    spark.readStream
    .format("kafka")
    .options(**kafka_options)
    .load()
)

json_stream_df = streaming_df.selectExpr("CAST(value AS STRING) AS json_value")
print("Event was writen successfully")

query = (
    json_stream_df.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", checkpoint_path)
    .start(bronze_path)
)
print("Events are being processed")

query.awaitTermination()
